to access the openAI API, you need an API key which you can get it from  https://platform.openai.com/api-keys
once you have a key then you can either save it as an environment variable or hard code in as needed from simplicity

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"



True

In [18]:
import os

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


Use document loaders to load data from a source as Document

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [19]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


you can also load directory using directory loader.
and split a long document into smaller chunks that can fit into your model's context window.

In [20]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


raw_documents = DirectoryLoader('../article', glob="*.pdf", loader_cls=PyPDFLoader).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)



Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings


vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()


invoke question

In [23]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about ramda AI"))

('Ramda AI is an innovative artificial intelligence (AI) platform that '
 'combines advanced technologies to help businesses enhance their operations, '
 'make better decisions, and improve customer experiences. It leverages '
 'state-of-the-art machine learning algorithms, natural language processing '
 '(NLP) techniques, and predictive analytics to enable organizations to derive '
 'valuable insights from their data, leading to informed decision-making.\n'
 '\n'
 'Key features of Ramda AI include:\n'
 '\n'
 '1. **Machine Learning Algorithms**: The platform uses cutting-edge machine '
 'learning to analyze data, identify patterns, and make accurate predictions. '
 'It learns continuously from new data, allowing it to adapt to changing '
 'business environments.\n'
 '\n'
 '2. **Natural Language Processing (NLP)**: Ramda AI incorporates NLP to '
 'understand and interpret unstructured data such as text, speech, and images. '
 'This capability enables businesses to extract meaningful inf